In [94]:
import requests
from bs4 import BeautifulSoup 
import csv
import pprint
import time

In [95]:
def crawl_laptop_data(URL):
    
    laptop_data = {'URL': URL}
    respone = requests.get(URL)

    try:
        if respone.status_code != 200:
            print(f"Failed to retrieve the {URL}")
            return None
        else:
            # create a soup object that make the HTMl searchable
            soup = BeautifulSoup(respone.content, 'html.parser')
            
            # get the product name
            product_name_element = soup.find('div', class_='box-product-name')
            if product_name_element:
                laptop_data['Name'] = product_name_element.text.strip()
            else:
                laptop_data['Name'] = 'Not Found'
            
            # get the price (notice that we only use sale price for this project)
            price_element = soup.find('div', class_='sale-price')
            if not price_element:
                price_element = soup.find('div', class_='box-price-present')
            if price_element:
                laptop_data['Price'] = price_element.text.strip()
            else:
                laptop_data['Price'] = 'Not Found'
            
            # get the "Thông Số Kỹ Thuật"    
            # extract the technical specs
            tech_info_container = soup.find('div', class_ = 'cps-block-technicalInfo')

            if tech_info_container:
                # find all the rows (<tr>) with the class 'technical-content-item'
                spec_rows = tech_info_container.find_all('tr', class_='technical-content-item')
                for row in spec_rows:
                    
                    # in each row, find the two cell (<td>)
                    cells = row.find_all('td')
                    if True:
                        spec_label = cells[0].text.strip()
                        spec_value = cells[1].text.strip()
 

                        if 'Loại CPU' in spec_label:
                            laptop_data['CPU'] = spec_value
                        elif 'Dung lượng RAM' in spec_label:
                            laptop_data['RAM'] = spec_value
                        elif 'Loại card đồ họa' in spec_label:
                            laptop_data['GPU'] = spec_value
                        elif 'Ổ cứng' in spec_label:
                            laptop_data['Storage'] = spec_value
                        elif 'Kích thước màn hình' in spec_label:
                            laptop_data['Screen Size'] = spec_value
                        elif 'Độ phân giải màn hình' in spec_label:
                            laptop_data['Screen Resolution'] = spec_value
                        elif 'Chất liệu tấm nền' in spec_label:
                            laptop_data['Panel'] = spec_value
                        elif 'Tần số quét' in spec_label:
                            laptop_data['Screen frequency'] = spec_value
                        elif 'Trọng lượng' in spec_label:
                            laptop_data['Weight'] = spec_value
                            
                return laptop_data
            
    except Exception as e:
        print(f"Something wrong with {URL}")
        print(e)
        return None

In [ ]:



CATEGORY_URL = 'https://cellphones.com.vn/laptop/asus.html'

category_respone = requests.get(CATEGORY_URL)
if category_respone.status_code != 200:
    print("Failed to connect")
else:
    category_soup = BeautifulSoup(category_respone.content, 'html.parser')
    product_links = []

    link_element = category_soup.find_all('a', class_='product__link')

    for link in link_element:
        # get the href attribute
        href = link.get('href')
        # make sure that this link does not exist in the list to avoid repeat
        if href and href not in product_links:
            product_links.append(href)
        
all_laptops_data = []

for link in product_links:
    data = crawl_laptop_data(link)
    if data:
        all_laptops_data.append(data)
   
    # stop for 1 second to not overload the server 
    time.sleep(1)
    


In [100]:
for link in link_element:
    print(link.get('href'))

https://cellphones.com.vn/laptop-asus-tuf-gaming-f16-fx607vj-rl034w.html
https://cellphones.com.vn/laptop-asus-vivobook-go-14-e1404fa-nk177w.html
https://cellphones.com.vn/laptop-asus-vivobook-16x-k3605vc-rp431w.html
https://cellphones.com.vn/laptop-asus-gaming-v16-v3607vu-rp343w.html
https://cellphones.com.vn/laptop-asus-vivobook-15-x1502va-bq885w.html
https://cellphones.com.vn/laptop-asus-tuf-gaming-f16-fx607vu-rl045w.html
https://cellphones.com.vn/laptop-asus-tuf-gaming-a15-fa506ncr-hn097w.html
https://cellphones.com.vn/laptop-asus-gaming-v16-v3607vm-rp044w.html
https://cellphones.com.vn/laptop-asus-vivobook-go-15-e1504ga-bq1141w.html
https://cellphones.com.vn/laptop-asus-vivobook-15-oled-a1505va-l1688w.html
https://cellphones.com.vn/laptop-asus-vivobook-15-oled-a1505va-ma468w.html
https://cellphones.com.vn/laptop-asus-vivobook-s14-s3407va-ly046w.html
https://cellphones.com.vn/laptop-asus-tuf-gaming-a17-fa706nfr-hx025w.html
https://cellphones.com.vn/laptop-asus-vivobook-16x-k3605vc-

In [98]:
# store it to csv file
csv_column = ['Name', 'Price', 'CPU', 'RAM', 'GPU','Storage', 'Screen Size', 'Screen Resolution', 'Panel', 'Screen frequency', 'Weight', 'URL']
csv_file = 'laptops_data_cellphones.csv'
try:
    with open(csv_file, 'w', newline='', encoding='utf-8') as file:
        writer = csv.DictWriter(file, fieldnames=csv_column)

        # ghi dòng tiêu đề
        writer.writeheader()

        # ghi dữ liệu của từng laptop 
        
        for data in all_laptops_data:
            writer.writerow(data) 

            
    print(f"store to file {csv_file}")
except IOError:
    print(f"Can not write to csv")

store to file laptops_data_cellphones.csv
